# OIB to Ome-Zarr (Linescan Confocal Microscopy) Batch Conversion

This script performs batch conversion of all `.oib` files in a folder to the Ome-Zarr format.


## Define directory containing the files to be converted

In [10]:
data_path = '/home/jovyan/LNMA/guerreroa/data'
data_folder = '20241114'

## Install and load required libraries

In [13]:
print("Installing necessary libraries...")
!pip install natsort oiffile ome-zarr > /dev/null 2>&1
print("Libraries installed successfully.")

Installing necessary libraries...
Libraries installed successfully.


## Import Libraries

In [20]:
import oiffile
import os
import zarr
from ome_zarr.io import parse_url
from natsort import natsorted
import numpy as np

## Functions

In [8]:
def clean_filenames(directory_path):
    """
    Cleans filenames in the specified directory by replacing certain characters
    and renaming the files in the filesystem.

    Parameters:
        directory_path (str): Path to the directory containing the files.

    Returns:
        list: A list of cleaned file names sorted naturally.
    """
    # Initialize a list to store cleaned file names
    cleaned_file_names = []

    # Process each file in the specified directory
    for file_name in os.listdir(directory_path):
        # Construct the original full path
        original_path = os.path.join(directory_path, file_name)

        # Clean the filename by replacing spaces and other unwanted characters
        cleaned_name = file_name.replace(' ', '_').replace('-', '_').replace('/', '_')
        cleaned_name = cleaned_name.replace('+', '_').replace('_copy', '')

        # Construct the new full path
        cleaned_path = os.path.join(directory_path, cleaned_name)

        # Rename the file if the cleaned name is different from the original name
        if original_path != cleaned_path:
            os.rename(original_path, cleaned_path)
            #print(f"Renamed {original_path} to {cleaned_path}")

        # Append the cleaned file name to the list
        cleaned_file_names.append(cleaned_name)

    # Return the list of cleaned file names, sorted naturally
    return natsorted(cleaned_file_names)

## Clean file names

In [15]:
directory_path = os.path.join(data_path, data_folder)
cleaned_files = clean_filenames(directory_path)
cleaned_files

['14_11_24_Exp1_TYH_FB_22C.oib', '14_11_24_Exp1_TYH_mib_22C_15segundos.oib']

In [18]:
directory_path

'/home/jovyan/LNMA/guerreroa/data/20241114'

## Create OME-Zarr files with metadata and rois included

In [22]:
for oib_file in cleaned_files:
    oib_file_path = os.path.join(directory_path, oib_file)
    ome_zarr_path = f'{oib_file_path[:-4]}.zarr'
    with oiffile.OifFile(oib_file_path) as oib:
        image_data = oib.asarray()  # Load image data
        all_metadata = dict(oib.mainfile)  # Retrieve all metadata as a dictionary
    
    ## Format Image Data for OME-Zarr
    
    # Ensure data shape is in (T, C, Z, Y, X) format
    while image_data.ndim < 5:
        image_data = np.expand_dims(image_data, axis=0)  # Add dimensions as needed

    # Convert the data to a numpy array (if not already)
    image_data = np.asarray(image_data)

    # Reorder data
    image_data = image_data.transpose(0, 2, 1, 3, 4)
    
    ## Remove Existing Zarr Store (if exists)
    if os.path.exists(ome_zarr_path):
        import shutil
        shutil.rmtree(ome_zarr_path)
        
    ## Save Image Data and Metadata to OME-Zarr
    store = parse_url(ome_zarr_path, mode="w").store
    root = zarr.group(store=store)
    dataset = root.create_dataset("image_data", data=image_data, chunks=True)

    # Save image metadata and ROI metadata as attributes
    root.attrs["image_metadata"] = all_metadata

print("Conversion complete. The OME-Zarr files are saved with all metadata.")


Conversion complete. The OME-Zarr files are saved with all metadata.
